In [1]:
#import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn import metrics

In [2]:
#Reading dataset
df=pd.read_csv('Hybrid.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Cate_name          68 non-null     object 
 1   Cate_HOMO          68 non-null     float64
 2   Cate_LUMO          68 non-null     float64
 3   Cate_MC_O1         68 non-null     float64
 4   Cate_MC_H1         68 non-null     float64
 5   Cate_MC_O2         68 non-null     float64
 6   Cate_MC_H2         68 non-null     float64
 7   Cate_BL_ArO1       68 non-null     float64
 8   Cate_BL_O1H1       68 non-null     float64
 9   Cate_BL_ArO2       68 non-null     float64
 10  Cate_BL_O2H2       68 non-null     float64
 11  Cate_total_dipole  68 non-null     float64
 12  Cate_RDKit_PC_0    68 non-null     float64
 13  Cate_RDKit_PC_1    68 non-null     float64
 14  Cate_RDKit_PC_2    68 non-null     float64
 15  Cate_RDKit_PC_3    68 non-null     float64
 16  Cate_RDKit_PC_4    68 non-nu

In [3]:
#Building descriptors
X = df.drop(columns=['Yield', 'Cate_name', 'P_name' ])
print('---Descriptors---')
print(X.head())

y = pd.DataFrame(df['Yield'],columns=['Yield'])
print('---Objective---')
print(y.head())

---Descriptors---
   Cate_HOMO  Cate_LUMO  Cate_MC_O1  Cate_MC_H1  Cate_MC_O2  Cate_MC_H2  \
0  -5.940433  -0.165443   -0.374537    0.264452   -0.416200    0.277039   
1  -5.940433  -0.165443   -0.374537    0.264452   -0.416200    0.277039   
2  -5.940433  -0.165443   -0.374537    0.264452   -0.416200    0.277039   
3  -5.940433  -0.165443   -0.374537    0.264452   -0.416200    0.277039   
4  -5.807916  -0.157824   -0.376813    0.263887   -0.418652    0.276114   

   Cate_BL_ArO1  Cate_BL_O1H1  Cate_BL_ArO2  Cate_BL_O2H2  ...    P_HOMO  \
0       1.36318       0.96622       1.37543       0.96253  ... -8.171191   
1       1.36318       0.96622       1.37543       0.96253  ... -8.457723   
2       1.36318       0.96622       1.37543       0.96253  ... -7.460440   
3       1.36318       0.96622       1.37543       0.96253  ... -8.082211   
4       1.36386       0.96625       1.37631       0.96239  ... -8.171191   

     P_LUMO    P_MC_P   P_MC_=O    P_MC_H  P_BL_P=O   P_BL_PH  P_total_dip

In [4]:
#storage 
data_r2_train =[]
data_RMSE_train = []
data_MAE_train = []
data_r2_test = []
data_RMSE_test = []
data_MAE_test = []
best_model_parametors = []
data_cv_score = []

#SVM_a_X,log(y)
for i in range(10):
    seed=i
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)
    
    #autoscaling
    a_X_train = (X_train - X_train.mean(axis=0)) / X_train.std(axis=0, ddof=1)
    a_X_test = (X_test - X_train.mean(axis=0)) / X_train.std(axis=0, ddof=1)
    
    log_y_train = np.log(y_train)
    
    param_svm={'C':[0.1,1,5,10,50,100,200,300,400,500,750,1000],
               'gamma':[100,10,1,0.1,0.01,0.001,0.0001,0.00001]}
    reg_svm = GridSearchCV(SVR(kernel='rbf', epsilon=0.3), param_grid=param_svm, cv=5, n_jobs=16)
    reg_svm.fit(a_X_train,log_y_train['Yield'])
    reg_best = reg_svm.best_estimator_
    y_pred1 = np.exp(reg_best.predict(a_X_train))
    y_pred2 = np.exp(reg_best.predict(a_X_test))
    
    #train
    r2_train = metrics.r2_score(y_train, y_pred1)
    RMSE_train = metrics.root_mean_squared_error(y_train, y_pred1)
    MAE_train =  metrics.mean_absolute_error(y_train, y_pred1)
    #test
    r2_test = metrics.r2_score(y_test, y_pred2)
    RMSE_test = metrics.root_mean_squared_error(y_test, y_pred2)
    MAE_test = metrics.mean_absolute_error(y_test, y_pred2)
    
    parametors = reg_svm.best_params_
    best_model_parametors.append(parametors)
    cv_score = reg_svm.best_score_
    data_cv_score.append(cv_score)
    
    data_r2_train.append(r2_train)
    data_RMSE_train.append(RMSE_train)
    data_MAE_train.append(MAE_train)
    data_r2_test.append(r2_test)
    data_RMSE_test.append(RMSE_test)
    data_MAE_test.append(MAE_test)
    
    print('----------------------')
    print('seed:', seed)
    print("Best Model Parameter:",reg_svm.best_params_)
    print("Best Model Score:",reg_svm.best_score_)
    print('R2_test:', r2_test)
print('R2_train_means:', sum(data_r2_train)/10)
print('CV_score_means:', sum(data_cv_score)/10)
print('R2_test_means:', sum(data_r2_test)/10)

----------------------
seed: 0
Best Model Parameter: {'C': 300, 'gamma': 0.001}
Best Model Score: 0.9082138328888988
R2_test: 0.7699594914988325
----------------------
seed: 1
Best Model Parameter: {'C': 400, 'gamma': 0.001}
Best Model Score: 0.8902179512438677
R2_test: 0.7930767955396595
----------------------
seed: 2
Best Model Parameter: {'C': 400, 'gamma': 0.001}
Best Model Score: 0.8931254563683124
R2_test: 0.8524694316076606
----------------------
seed: 3
Best Model Parameter: {'C': 300, 'gamma': 0.001}
Best Model Score: 0.8743217133852484
R2_test: 0.886367387894889
----------------------
seed: 4
Best Model Parameter: {'C': 300, 'gamma': 0.001}
Best Model Score: 0.8955396175849362
R2_test: 0.8347676765230324
----------------------
seed: 5
Best Model Parameter: {'C': 1000, 'gamma': 0.0001}
Best Model Score: 0.8584041420207594
R2_test: 0.5166467087748308
----------------------
seed: 6
Best Model Parameter: {'C': 200, 'gamma': 0.001}
Best Model Score: 0.8713579987955871
R2_test: 0.7

In [5]:
data_r2_train_pd = pd.DataFrame(data= data_r2_train, columns=['r2_train'])
data_RMSE_train_pd = pd.DataFrame(data=data_RMSE_train, columns=['RMSE_train'])
data_MAE_train_pd = pd.DataFrame(data=data_MAE_train, columns=['MAE_train'])
data_r2_test_pd = pd.DataFrame(data=data_r2_test, columns=['r2_test'])
data_RMSE_test_pd = pd.DataFrame(data=data_RMSE_test, columns=['RMSE_test'])
data_MAE_test_pd = pd.DataFrame(data=data_MAE_test, columns=['MAE_test'])
data_cv_score_pd = pd.DataFrame(data=data_cv_score, columns=['cv_score'])
data_parametors = pd.DataFrame([best_model_parametors])

data_all = pd.concat([data_r2_train_pd, data_RMSE_train_pd, data_MAE_train_pd, data_r2_test_pd, 
                      data_RMSE_test_pd, data_MAE_test_pd, data_cv_score_pd],
                     axis=1, join='inner')

data_all.loc['mean'] = data_all.mean()
data_all.loc['std'] = data_all.std(ddof=0)
print(data_all)

data_all.to_csv('../../score/Hybrid//SVM_score.csv')

      r2_train  RMSE_train  MAE_train   r2_test  RMSE_test   MAE_test  \
0     0.899443    8.610282   5.587166  0.769959  17.345936  11.042409   
1     0.874430   10.656172   7.228373  0.793077  14.158995   8.564393   
2     0.904288    9.409723   6.201566  0.852469  11.944500   7.699992   
3     0.874453   10.821236   7.243534  0.886367  10.443105   6.458032   
4     0.910963    8.761198   5.187908  0.834768  13.255019   8.938799   
5     0.757530   14.012051   7.376752  0.516647  24.245747  14.781909   
6     0.875754   10.097792   6.411109  0.746898  17.452726  10.839789   
7     0.892654   10.132908   6.516067  0.805695  13.271736   9.146533   
8     0.784350   13.776705   7.744435  0.500591  22.215632  14.075818   
9     0.836604   12.868296   8.149509  0.850173  10.275057   6.568197   
mean  0.861047   10.914636   6.764642  0.755664  15.460845   9.811587   
std   0.047296    1.788045   0.856700  0.123590   4.328660   2.601471   

      cv_score  
0     0.908214  
1     0.890218  